# transcribe audio to text

In [ ]:
import pyaudio
import numpy as np
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Configuration
MODEL_NAME = "facebook/wav2vec2-base"
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # Wav2Vec2 expects 16kHz audio
CHUNK = 1024  # Number of audio samples per frame

# Load the Wav2Vec2 model and processor
print("Loading Wav2Vec2 model...")
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_NAME)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Initialize PyAudio
audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT, 
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

print("Recording and transcribing in real-time... Press Ctrl+C to stop.")

# Buffer for real-time processing
audio_buffer = np.zeros((RATE * 5,), dtype=np.int16)  # 5-second sliding window

try:
    while True:
        # Read audio data from the microphone
        data = stream.read(CHUNK)
        audio_samples = np.frombuffer(data, dtype=np.int16)

        # Add new samples to the buffer and keep only the last 5 seconds
        audio_buffer = np.concatenate((audio_buffer, audio_samples))[-RATE * 5:]

        # Normalize and prepare input features
        input_values = processor(
            audio_buffer, 
            sampling_rate=RATE, 
            return_tensors="pt", 
            padding=True
        ).input_values

        # Move input values to the appropriate device
        input_values = input_values.to(device)

        # Perform transcription
        with torch.no_grad():
            logits = model(input_values).logits
            predicted_ids = torch.argmax(logits, dim=-1)
            transcription = processor.decode(predicted_ids[0])

        # Print the transcription in real-time
        print(f"\rTranscription: {transcription}", end="")
except KeyboardInterrupt:
    print("\nTranscription stopped.")
finally:
    # Stop and clean up
    stream.stop_stream()
    stream.close()
    audio.terminate()
    print("\nRecording and transcription stopped.")


In [6]:
%pip install scipy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pyaudio
import numpy as np
import torch
from scipy.signal import resample
from pynput import keyboard
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Suppress TensorFlow and other warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

# Configuration
MODEL_NAME = "openai/whisper-tiny"  # Use a smaller model for faster transcription
FORMAT = pyaudio.paInt16
CHUNK = 2048  # Increased buffer size to prevent overflow
DEVICE_INDEX = 0  # Replace with the correct device index
RATE = 44100  # Default sample rate of your device
TARGET_RATE = 16000  # Whisper's required sample rate
CHANNELS = 1  # Use mono input for transcription

# Load Whisper model and processor
print("Loading Whisper model...")
processor = WhisperProcessor.from_pretrained(MODEL_NAME)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Initialize PyAudio
audio = pyaudio.PyAudio()
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    input_device_index=DEVICE_INDEX,
                    frames_per_buffer=CHUNK)

print(f"Using device '{audio.get_device_info_by_index(DEVICE_INDEX)['name']}' at {RATE} Hz.")
print("Recording and transcribing in real-time... Press 'S' to stop.")

# Real-time buffer
audio_buffer = np.zeros((TARGET_RATE * 5,), dtype=np.int16)  # 5-second sliding window
recording = True

# Handle key press to stop transcription
def on_press(key):
    global recording
    try:
        if key.char == 's':  # Stop on 'S' key press
            print("\n'S' key pressed. Stopping transcription...")
            recording = False
            return False
    except AttributeError:
        pass

listener = keyboard.Listener(on_press=on_press)
listener.start()

try:
    while recording:
        try:
            # Read audio data
            data = stream.read(CHUNK, exception_on_overflow=False)
            audio_samples = np.frombuffer(data, dtype=np.int16)

            # Resample from 44100 Hz to 16000 Hz
            resampled_audio = resample(audio_samples, int(len(audio_samples) * TARGET_RATE / RATE))

            # Add to audio buffer
            audio_buffer = np.concatenate((audio_buffer, resampled_audio))[-TARGET_RATE * 5:]

            # Normalize audio and process with Whisper
            input_features = processor(
                audio_buffer.astype(np.float32) / 32768.0,  # Normalize int16 to float32
                sampling_rate=TARGET_RATE,
                return_tensors="pt",
                language="en"  # Set language for transcription
            ).input_features

            input_features = input_features.to(device)
            with torch.no_grad():
                predicted_ids = model.generate(input_features)
            transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

            # Print the transcription
            print(f"\rTranscription: {transcription}", end="")
        except OSError as e:
            if e.errno == -9981:  # Input overflow error
                print("\nWarning: Input overflowed. Skipping this chunk.")
                continue
except Exception as e:
    print(f"\nError: {e}")
finally:
    # Cleanup
    stream.stop_stream()
    stream.close()
    audio.terminate()
    listener.join()
    print("\nTranscription stopped.")


Loading Whisper model...
Using device 'HDA Intel PCH: ALC897 Analog (hw:0,0)' at 44100 Hz.
Recording and transcribing in real-time... Press 'S' to stop.
Transcription:  You not sure if you can see it.-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-S-아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아 아

In [ ]:
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
